In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.recurrent import LSTM #This is giving an error while importing 
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization,Conv1D,MaxPooling1D
# from tensorflow.keras import layers # Added due to error in recurrent layer import
from keras.models import Sequential, load_model, save_model
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

AttributeError: module 'tensorflow.core.framework.types_pb2' has no attribute 'SerializedDType'

### Loading Dataset

In [ ]:
df = pd.read_csv("my_data.csv")
plot_x = df['Date'].copy()
df.set_index("Date", inplace = True)
df.index = pd.to_datetime(df.index)
df.head()
# print(df.columns.tolist())

In [ ]:
data=[{
    "Open":1629.70,
    "Close": 160.00,
    "High":1631.70,
    "Low":114.10,
    "Volume": 1692315,
    "Date" : 25-11-2022
}]

In [ ]:
df=pd.DataFrame(data)
print(df)
df.set_index("Date", inplace = True)
df.index = pd.to_datetime(df.index)
df.head()

In [ ]:
print(type(plot_x))

### Predicting Prices

In [ ]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x = df[['Open', 'Low', 'High', 'Volume']].copy()
y = df['Close'].copy()

x [['Open', 'Low', 'High', 'Volume']] = x_scaler.fit_transform(x)
y = y_scaler.fit_transform(y.values.reshape(-1, 1))

In [ ]:
def load_data(X, seq_len, train_size=0.9):
    amount_of_features = X.shape[1]
    X_mat = X.values
    sequence_length = seq_len + 1
    data = []
    
    for index in range(len(X_mat) - sequence_length):
        data.append(X_mat[index: index + sequence_length])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    x_test = data[train_split:, :-1] 
    y_test = data[train_split:, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return x_train, y_train, x_test, y_test

In [ ]:
window = 22
x['close'] = y
X_train, y_train, X_test, y_test = load_data(x, window)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print (X_train[0], y_train[0])

In [ ]:
def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="uniform",activation='relu'))        
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = build_model(input_shape=(window, 5))

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1)

In [ ]:
save_model(model, "model.h5")

In [ ]:
model = load_model("model.h5")

In [ ]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

In [ ]:
trainPredict = y_scaler.inverse_transform(trainPredict)
trainY = y_scaler.inverse_transform([y_train])
testPredict = y_scaler.inverse_transform(testPredict)
testY = y_scaler.inverse_transform([y_test])

In [ ]:
type(testPredict)

In [ ]:
plot_predicted = testPredict.copy()
# print(plot_predicted.shape)
# plot_predicted = plot_predicted.reshape(145, 1)
plot_actual = testY.copy()
plot_actual = plot_actual.reshape(243, 1)
print(plot_actual.shape)
# print(plot_predicted.shape)

In [ ]:
plot_x = pd.to_datetime(plot_x.iloc[-174:])

In [ ]:
plt.plot(pd.DataFrame(plot_predicted), label='Predicted')
plt.plot(pd.DataFrame(plot_actual), label='Actual')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(pd.DataFrame(plot_actual), label='Actual')
plt.legend(loc='best')
plt.show()

In [ ]:
trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
print('Train Score: %.2f RMSE' % (trainScore))
testScore = metrics.mean_squared_error(testY[0], testPredict[:,0]) ** .5
print('Test Score: %.2f RMSE' % (testScore))

### Plotting Results

In [ ]:
nifty_stock_prices = df["Close"].values.astype('float32')
nifty_stock_prices = nifty_stock_prices.reshape(len(nifty_stock_prices), 1)

In [ ]:
trainPredictPlot = np.empty_like(nifty_stock_prices)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[window:len(trainPredict)+window, :] = trainPredict

testPredictPlot = np.empty_like(nifty_stock_prices)
testPredictPlot[:, :] = np.nan
testPredictPlot[(len(nifty_stock_prices) - testPredict.shape[0]):len(nifty_stock_prices), :] = testPredict

In [ ]:
plt.plot(pd.DataFrame(nifty_stock_prices, columns=["Close"], index=df.index).Close, label='Actual')
plt.plot(pd.DataFrame(trainPredictPlot, columns=["Close"], index=df.index).Close, label='Training')
plt.plot(pd.DataFrame(testPredictPlot, columns=["Close"], index=df.index).Close, label='Testing')
plt.legend(loc='best')
plt.title("NIFTY50")
plt.show()